# Welcome to our NLP Workshop!

![google search NLP](https://miro.medium.com/max/1698/1*Xo-IYQR0kj_JHFUmwS2Oig.png)

We followed this [wonderful article](https://towardsdatascience.com/next-word-prediction-with-nlp-and-deep-learning-48b9fe0a17bf) by Bharath K 

In [ ]:
# import libraries 
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os


In [ ]:
visual = open("prideandprejudice_clean.txt", "r", encoding = "utf8")
print(visual.read())
visual.close

﻿The Project Gutenberg eBook of Pride and Prejudice, by Jane Austen

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: Pride and Prejudice

Author: Jane Austen

Release Date: June, 1998 [eBook #1342]
[Most recently updated: February 10, 2021]

Language: English

Character set encoding: UTF-8

Produced by: Anonymous Volunteers and David Widger

*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***

THERE IS AN ILLUSTRATED EDITION OF THIS TITLE WHICH MAY VIEWED AT EBOOK
[# 42671 ]

cover




      Pride and Prejudice

      By Jane Austen

        CONTENTS

         Chapter 1

      

<function TextIOWrapper.close()>

## Pre-processing 

In [ ]:
import string

file = open("prideandprejudice_clean.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
    
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) # map punctuation to space
new_data = data.translate(translator)

z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)


In [ ]:
data[1000:3000] # without space mapping

'be want wife. However little known feelings views such on his first entering neighbourhood, so well fixed minds surrounding families, he considered rightful property some one their daughters. “My dear Mr. Bennet,” said lady him day, “have heard Netherfield Park let last?” Bennet replied had not. “But is,” returned she; “for Mrs. Long has just been here, she told me all about it.” made answer. “Do know who taken it?” cried wife impatiently. “_You_ tell me, I objection hearing was invitation enough. “Why, my dear, know, says young large fortune from north England; came down Monday chaise four see place, much delighted agreed Morris immediately; take Michaelmas, servants house end next week.” “What name?” “Bingley.” “Is married single?” “Oh! Single, sure! A fortune; five thousand year. What fine thing our girls!” “How so? How can affect them?” wife, “how tiresome! am thinking marrying them.” design settling here?” “Design! Nonsense, how talk so! But very likely _may_ fall love them, ther

In [ ]:
new_data[1000:3000] # with space mapping

'  Chapter 2           Chapter 3           Chapter 4           Chapter 5           Chapter 6           Chapter 7           Chapter 8           Chapter 9           Chapter 10           Chapter 11           Chapter 12           Chapter 13           Chapter 14           Chapter 15           Chapter 16           Chapter 17           Chapter 18           Chapter 19           Chapter 20           Chapter 21           Chapter 22           Chapter 23           Chapter 24           Chapter 25           Chapter 26           Chapter 27           Chapter 28           Chapter 29           Chapter 30           Chapter 31           Chapter 32           Chapter 33           Chapter 34           Chapter 35           Chapter 36           Chapter 37           Chapter 38           Chapter 39           Chapter 40           Chapter 41           Chapter 42           Chapter 43           Chapter 44           Chapter 45           Chapter 46           Chapter 47           Chapter 48           Chapter 49        

## Tokenizer
Reference:
- [Keras Tokenizer Docs](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer)
- [Keras pre-processing source code](https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/text.py) 

In [ ]:
tokenizer = Tokenizer() # from keras
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]

sizeOV = len(tokenizer.word_index) + 1

sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
sequences = np.array(sequences)

X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

y = to_categorical(y, num_classes=sizeOV)

In [ ]:
print(sequences) # to visualize the arrays

[[ 328  525]
 [ 525   16]
 [  16  329]
 ...
 [7246 2830]
 [2830 7247]
 [7247 1502]]


## Creating the model

Refrence:
- https://keras.io/api/layers/core_layers/embedding/
- https://keras.io/api/layers/recurrent_layers/lstm/
- https://deepai.org/machine-learning-glossary-and-terms/recurrent-neural-network 

![LSTM Graphic](https://colah.github.io/images/post-covers/lstm.png)

![](https://www.mdpi.com/electronics/electronics-09-00289/article_deploy/html/images/electronics-09-00289-g006.png)

In [ ]:
# Importing necessary libraries
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense

In [ ]:
model = Sequential()
model.add(Embedding(sizeOV, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(sizeOV, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             72480     
_________________________________________________________________
lstm (LSTM)                  (None, 1, 1000)           4044000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense (Dense)                (None, 1000)              1001000   
_________________________________________________________________
dense_1 (Dense)              (None, 7248)              7255248   
Total params: 20,376,728
Trainable params: 20,376,728
Non-trainable params: 0
_________________________________________________________________


## Using Callbacks

In [ ]:
# Importing necessary libraries
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reducer = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

## Compile and fit

In [ ]:
# model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))
# model.fit(X, y, epochs=50, batch_size=64, callbacks=[checkpoint, reducer])

Epoch 1/50
237/237 [==============================] - 60s 242ms/step - loss: 7.4001

Epoch 00001: loss improved from 7.68640 to 7.52799, saving model to nextword1.h5
Epoch 2/50
237/237 [==============================] - 57s 242ms/step - loss: 7.1153

Epoch 00002: loss improved from 7.52799 to 7.12582, saving model to nextword1.h5
Epoch 3/50
237/237 [==============================] - 57s 243ms/step - loss: 6.7376

Epoch 00003: loss improved from 7.12582 to 6.75397, saving model to nextword1.h5
Epoch 4/50
237/237 [==============================] - 57s 242ms/step - loss: 6.3779

Epoch 00004: loss improved from 6.75397 to 6.40108, saving model to nextword1.h5
Epoch 5/50
237/237 [==============================] - 57s 243ms/step - loss: 6.0078

Epoch 00005: loss improved from 6.40108 to 6.05718, saving model to nextword1.h5
Epoch 6/50
237/237 [==============================] - 57s 242ms/step - loss: 5.6459

Epoch 00006: loss improved from 6.05718 to 5.74632, saving model to nextword1.h5
Epoc

In [ ]:
# model.save('nlp_model.h5')

In [ ]:
model = keras.models.load_model('nlp_model.h5')

## Using our model

In [ ]:
def Predict_Next_Words(model, tokenizer, text):
    for i in range(3):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = np.array(sequence)
        
        preds = model.predict_classes(sequence)
        predicted_word = ""
        
        for key, value in tokenizer.word_index.items():
            if value == preds:
                predicted_word = key
                break
        
        print(predicted_word)
        return predicted_word

In [ ]:
while(True):

    text = input("Enter your line: ")
    
    if text == "STSP":
        print("Ending The Program.....")
        break
    
    else:
        try:
            text = text.split(" ")
            text = text[-1]

            text = ''.join(text)
            Predict_Next_Words(model, tokenizer, text)

            
            
        except:
            continue

## Possible improvement to the model

- Lowercase all the text
- Remove all non-alphabetical characters
- Use multiple embedding layers to extract different representations of words
- Take out beginning of book (table of contents, acknowledgments, copyright statements)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fd9a6c66-9236-41d2-811a-b5672d2b4284' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>